In [8]:
import pandas as pd
import numpy as np
import os

df = pd.read_csv("../Feynman_with_units/I.6.2", 
                 sep=r"\s+",  # whitespace separator
                 header=None).head(1000) # no header row
print(df.shape)

(1000, 3)


In [9]:
df.head()

,0,1,2
0,2.564830,1.066371,0.142664
1,2.893217,2.871331,0.084266
2,2.315051,1.634639,0.134303
3,1.429826,2.852188,0.038156
4,2.293785,2.828017,0.081336


In [10]:
def batch_encode_p1000(numbers):
    """
    Encode a batch of numbers using the P1000 scheme.
    
    Parameters:
      numbers (np.ndarray): Array of numbers to encode.
      
    Returns:
      sign_tokens (np.ndarray): Array of sign tokens ('+' or '-').
      mantissa_tokens (np.ndarray): Array of mantissa tokens as zero-padded strings.
      exponent_tokens (np.ndarray): Array of exponent tokens as strings (e.g., 'E+3').
    """
    # Convert input to numpy array
    numbers = np.array(numbers, dtype=float)  # Ensure floating-point
    
    # Sign tokens: vectorized assignment based on the sign of the numbers
    sign_tokens = np.where(numbers < 0, '-', '+')
    
    # Work with absolute values
    abs_numbers = np.abs(numbers)
    
    # Handle zeros separately to avoid log10 issues
    non_zero = abs_numbers > 0
    exponent = np.zeros_like(abs_numbers, dtype=int)
    normalized = np.zeros_like(abs_numbers, dtype=float)
    
    # Compute exponent where numbers are non-zero
    exponent[non_zero] = np.floor(np.log10(abs_numbers[non_zero])).astype(int)
    
    # Compute normalized values for non-zero numbers - using float powers to avoid integer power error
    normalized[non_zero] = abs_numbers[non_zero] / np.power(10.0, exponent[non_zero])
    
    # Scale the normalized values to get mantissa in [0, 1000)
    # Multiply by 100 since normalized values are in [1, 10)
    mantissas = np.round(normalized * 100).astype(int)
    
    # Correct any mantissa that rounds to 1000
    overflow = mantissas >= 1000
    if np.any(overflow):
        mantissas[overflow] = 100
        exponent[overflow] += 1
    
    # Format mantissas as 3-digit strings
    mantissa_tokens = np.array([f"{m:03d}" for m in mantissas])
    
    # Format exponent tokens as strings (e.g., 'E+3' or 'E-2')
    exponent_tokens = np.array([f"E{'+' if e >= 0 else ''}{e}" for e in exponent])
    
    return sign_tokens, mantissa_tokens, exponent_tokens




    
    


In [11]:


# Assuming batch_encode_p1000 function is already defined above

def process_file_with_columns(file_path, col_sep_token="[COL_SEP]"):
    """
    Process a single CSV file containing numerical data.
    Encodes each number using the P1000 scheme and preserves column structure by
    inserting a delimiter token between columns.
    
    Returns a list of tokens for the entire file.
    """
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path,sep=r"\s+",header=None).head(1000)
    
    # We'll build a token list row by row
    token_list = []
    
    # Iterate over each row in the DataFrame
    for idx, row in df.iterrows():
        row_tokens = []
        # Process each column in the row separately
        for col in df.columns:
            # Encode the number in this cell; each cell may be a single number.
            # If a cell contains multiple numbers, you might need to adjust this.
            number = row[col]
            # For a single number, get the tokens as a list
            sign_tokens, mantissa_tokens, exponent_tokens = batch_encode_p1000([number])
            # Combine the triplet (they are arrays with one element each)
            number_tokens = [sign_tokens[0], mantissa_tokens[0], exponent_tokens[0]]
            row_tokens.extend(number_tokens)
            # Insert column delimiter after each column (except last column)
            if col != df.columns[-1]:
                row_tokens.append(col_sep_token)
        # Optionally, you can add a row delimiter (e.g., [ROW_SEP]) if needed.
        token_list.extend(row_tokens)
        token_list.append("[ROW_SEP]")
    
    return token_list

# # Example usage for a single file:
# file_path = "../Feynman_with_units/I.6.2"  # Update the path as needed
# token_list = process_file_with_colum                                                                                                                   ns(file_path)
# print(token_list[:20])  # Print first 20 tokens for inspection
# print("Total tokens in file:", len(token_list))


In [13]:
folder = "../Feynman_with_units"

file_list = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
print(file_list)

['II.8.31', 'III.12.43', 'I.18.4', 'II.10.9', 'II.11.27', 'III.21.20', 'I.18.14', 'III.9.52', 'I.6.2', 'II.36.38', 'II.2.42', 'II.13.34', 'I.13.12', 'III.4.33', 'I.12.5', 'I.48.20', 'II.34.29a', 'I.34.27', 'I.34.14', 'II.13.23', 'I.50.26', 'II.4.23', 'III.15.12', 'II.21.32', 'I.6.2b', 'II.11.20', 'II.11.7', 'I.39.11', 'II.6.15a', 'I.32.17', 'I.14.4', 'I.15.3x', 'II.27.18', 'I.6.2a', 'I.43.31', 'I.12.4', 'III.7.38', 'III.17.37', 'I.15.3t', 'III.10.19', 'II.6.11', 'I.32.5', 'I.39.1', 'I.34.8', 'I.13.4', 'II.34.2a', 'I.34.1', 'II.34.2', 'II.3.24', 'I.29.16', 'I.12.1', 'I.43.16', 'I.47.23', 'I.11.19', 'I.12.2', 'II.37.1', 'I.30.5', 'II.34.29b', 'I.40.1', 'II.11.3', 'III.13.18', 'I.39.22', 'II.24.17', 'I.26.2', 'I.24.6', 'II.38.3', 'III.15.27', 'I.25.13', 'III.14.14', 'II.27.16', 'III.15.14', 'II.35.18', 'I.15.10', 'I.14.3', 'I.37.4', 'I.38.12', 'III.4.32', 'III.8.54', 'I.18.12', 'II.15.4', 'I.12.11', 'I.44.4', 'I.41.16', 'II.15.5', 'II.35.21', 'II.38.14', 'I.9.18', 'I.43.43', 'I.16.6', 'II

In [16]:
print(len(file_list))

100


In [15]:
def process_all_files_with_columns(file_list,folder_path, col_sep_token="[COL_SEP]"):
    all_file_tokens = {}
    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)
        tokens = process_file_with_columns(file_path, col_sep_token)
        all_file_tokens[file_name] = tokens
        print(f"Processed {file_name}, token sequence length: {len(tokens)}")
    return all_file_tokens

# Example usage:
folder_path = "../Feynman_with_units"
all_tokens = process_all_files_with_columns(file_list,folder_path)


Processed II.8.31, token sequence length: 12000
Processed III.12.43, token sequence length: 12000
Processed I.18.4, token sequence length: 20000
Processed II.10.9, token sequence length: 16000
Processed II.11.27, token sequence length: 20000
Processed III.21.20, token sequence length: 20000
Processed I.18.14, token sequence length: 20000
Processed III.9.52, token sequence length: 28000
Processed I.6.2, token sequence length: 12000
Processed II.36.38, token sequence length: 36000
Processed II.2.42, token sequence length: 24000
Processed II.13.34, token sequence length: 16000
Processed I.13.12, token sequence length: 24000
Processed III.4.33, token sequence length: 20000
Processed I.12.5, token sequence length: 12000
Processed I.48.20, token sequence length: 16000
Processed II.34.29a, token sequence length: 16000
Processed I.34.27, token sequence length: 12000
Processed I.34.14, token sequence length: 16000
Processed II.13.23, token sequence length: 16000
Processed I.50.26, token sequenc

In [7]:
from tokenizers import Tokenizer

tokenizer=Tokenizer.from_file("my_tokenizer.json")

token_string=" ".join(token_list)

encoded=tokenizer.encode(token_string)

encoded_lines=[]
encoded_lines.append(encoded)
print(len(encoded_lines[0]))


12000


In [24]:
from tokenizers import Tokenizer

tokenizer=Tokenizer.from_file("my_tokenizer.json")
tokenized_dict={}
for key,value in all_tokens.items():
    current_token_string=" ".join(value)
    encoding_result = tokenizer.encode(current_token_string)  
    tokenized_dict[key] = encoding_result.ids


    

In [26]:
import json
with open("dataset_encoded.json", "w") as f:
    json.dump(tokenized_dict, f, indent=4)